In [ ]:
import warnings
from functions import *
from tqdm.auto import tqdm
import ray

In [ ]:
ray.init()

In [ ]:
T      = 100
#gamma3, gamma4 = -2.448, 10.164
gamma3, gamma4 = 0, 3
K = 10
N = 10_000
SR0 = 0
SR1 = .2
p_H1 = .30  # Should be sufficiently high, otherwise SR_c will be very high -- if H1 is rare/exceptional, we need very strong evidence to reject H0
q = .25

s0 = math.sqrt( sharpe_ratio_variance( SR0, T, gamma3=gamma3, gamma4=gamma4 ) )
s1 = math.sqrt( sharpe_ratio_variance( SR1, T, gamma3=gamma3, gamma4=gamma4 ) )

SR = SR0
V = sharpe_ratio_variance( SR, T, gamma3 = gamma3, gamma4 = gamma4 )
SRs = np.random.normal( SR, math.sqrt(V), (K, N) )
max_SR0s = SRs.max(axis=0)
SR0_adj = expected_maximum_sharpe_ratio( K, SRs.var(axis=0) )

SR = SR1
V = sharpe_ratio_variance( SR, T, gamma3 = gamma3, gamma4 = gamma4 )
SRs = np.random.normal( SR, math.sqrt(V), (K, N) )
max_SR1s = SRs.max(axis=0)
SR1_adj = expected_maximum_sharpe_ratio( K, SRs.var(axis=0) )

H1 = np.random.uniform(size=N) < p_H1
H0 = ~ H1
max_SRs = np.where( H1, max_SR1s, max_SR0s )
SR_adj = np.where( H1, SR1_adj, SR0_adj )

In [ ]:
SR_c_FDR = control_for_FDR( q, SR0 = SR0, SR1 = SR1, p_H1 = p_H1, T = T, gamma3 = gamma3, gamma4 = gamma4, K = K )[2]

@ray.remote
def f(u):
    return control_for_FDR( q, SR0 = SR0 + u, SR1 = SR1 + u, p_H1 = p_H1, T = T, gamma3 = gamma3, gamma4 = gamma4, K=K, grid_size = 100 )
SR_c_FWER_FDR = [ f.remote(u) for u in SR_adj ]
SR_c_FWER_FDR = [ ray.get(u) for u in tqdm(SR_c_FWER_FDR) ]
SR_c_FWER_FDR = np.array(SR_c_FWER_FDR)

In [ ]:
tmp = pd.DataFrame( SR_c_FWER_FDR, columns = ['alpha', 'beta', 'SR_c', 'q_hat'] )
SR_c_FWER_FDR = tmp['SR_c'].values

In [ ]:
fig, ax = plt.subplots( figsize = (5,3), layout = 'constrained', dpi = 300 )
ax.hist( max_SRs[H0], bins = 20, density = False, alpha = .4, label = 'H0' )
ax.hist( max_SRs[H1], bins = 20, density = False, alpha = .4, label = 'H1' )
ax.axvline( SR_c_FDR,             color = 'black', linestyle = '--' )
ax.axvline( SR_c_FWER_FDR.mean(), color = 'black', linestyle = '--' )
ax.axvline( (SR0+SR_adj[H0]).mean(), color = 'white', linestyle = ':', linewidth = 1)
ax.axvline( (SR1+SR_adj[H1]).mean(), color = 'white', linestyle = ':', linewidth = 1)
ax.set_xlabel('Sharpe ratio' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.legend()
plt.show()

In [ ]:
data = pd.DataFrame( { 
    'SR_c (FDR)':      np.ones(shape=N) * SR_c_FDR,
    'SR_c (FWER-FDR)': SR_c_FWER_FDR,
    'SR': max_SRs,
    'P_FDR':      max_SRs > SR_c_FDR,
    'P_FWER_FDR': max_SRs > SR_c_FWER_FDR,
    'H0': H0,
    'H1': H1,
} )

fig, axs = plt.subplots(1, 2, figsize = (8, 4), layout = 'constrained', dpi = 300 )
for x_label, y_label, ax in zip(
    ['SR_c (FDR)',  'SR_c (FWER-FDR)'],
    ['SR',          'SR' ],
    axs,
): 
    ax.scatter( data[y_label].max(), data[y_label].max(), alpha = 0 )  # To set xlim, ylim
    ax.scatter( data[x_label][H0],  data[y_label][H0],  color = 'tab:blue',   alpha = 1, label = 'H0' )
    ax.scatter( data[x_label][~H0], data[y_label][~H0], color = 'tab:orange', alpha = 1, label = 'H1' )
    ax.scatter( data[x_label], data[y_label], color = [ 'tab:orange' if h else 'tab:blue' for h in H1 ], alpha = .1 )
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.axline( (0,0), slope = 1, color = 'black', linestyle = ':', linewidth = 1 )

    #ax.axhline( SR0, color = 'black', linestyle = ':', linewidth = 1 )
    #ax.axhline( SR1, color = 'black', linestyle = ':', linewidth = 1 )

    #ax.axhline( SR0 + res['SR0_adj'].mean(), color = 'black', linestyle = ':', linewidth = 1 )
    #ax.axhline( SR1 + res['SR0_adj'].mean(), color = 'black', linestyle = ':', linewidth = 1 )

axs[0].set_title( f"FDR = {100*np.sum( H0 & data['P_FDR'] ) / np.sum( data['P_FDR'] ):.1f}%")    
axs[1].set_title( f"FDR = {100*np.sum( H0 & data['P_FWER_FDR'] ) / np.sum( data['P_FWER_FDR'] ):.1f}%")

axs[0].legend(loc='lower right')
plt.show()